<a href="https://colab.research.google.com/github/jwiggerthale/HiL-Machine-Learnig/blob/main/Datasplitforyolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub
from glob import glob
import os
import yaml

In [2]:
dataset_path = kagglehub.dataset_download("zhangyunsheng/defects-class-and-location")

label_files = glob(os.path.join(dataset_path, '**/*.xml'), recursive=True)
image_files = glob(os.path.join(dataset_path, '**/images/*/*.jpg'), recursive=True)  # Adjusted pattern
label_files[:5], image_files[:5]

100%|██████████| 923M/923M [00:12<00:00, 77.8MB/s]

Extracting files...


(['/root/.cache/kagglehub/datasets/zhangyunsheng/defects-class-and-location/versions/1/label/label/img_02_436153600_00669.xml',
  '/root/.cache/kagglehub/datasets/zhangyunsheng/defects-class-and-location/versions/1/label/label/img_06_425237000_00673.xml',
  '/root/.cache/kagglehub/datasets/zhangyunsheng/defects-class-and-location/versions/1/label/label/img_03_3403404300_01219.xml',
  '/root/.cache/kagglehub/datasets/zhangyunsheng/defects-class-and-location/versions/1/label/label/img_06_425608200_00007.xml',
  '/root/.cache/kagglehub/datasets/zhangyunsheng/defects-class-and-location/versions/1/label/label/img_06_3403399700_00746.xml'],
 ['/root/.cache/kagglehub/datasets/zhangyunsheng/defects-class-and-location/versions/1/images/images/waist folding/img_06_3436814600_00007.jpg',
  '/root/.cache/kagglehub/datasets/zhangyunsheng/defects-class-and-location/versions/1/images/images/waist folding/img_06_425614600_00423.jpg',
  '/root/.cache/kagglehub/datasets/zhangyunsheng/defects-class-and-l

In [ ]:
# from PIL import Image
# im = Image.open('/root/.cache/kagglehub/datasets/zhangyunsheng/defects-class-and-location/versions/1/images/images/waist folding/img_08_425501100_00051.jpg')
# print(im.size)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
!rm -rf /content/datanew1

In [4]:
class_map = {
    'crease': 0, 'crescent_gap': 1, 'inclusion': 2,
    'oil_spot': 3, 'punching_hole': 4, 'rolled_pit': 5,
    'silk_spot': 6, 'waist_folding': 7, 'water_spot': 8,
    'welding_line': 9
}



In [ ]:
# import os

# label_folder = '/content/drive/MyDrive/ims'

# for filename in os.listdir(label_folder):
#     if filename.endswith('.txt'):
#         file_path = os.path.join(label_folder, filename)
#         with open(file_path, 'r') as f:
#             lines = f.readlines()
#         new_lines = []
#         for line in lines:
#             parts = line.strip().split()
#             if parts:
#                 class_name = parts[0]
#                 if class_name in class_map:
#                     parts[0] = str(class_map[class_name])
#                 new_lines.append(' '.join(parts))
#         with open(file_path, 'w') as f:
#             f.write('\n'.join(new_lines))
# print('Conversion complete.')

In [ ]:
# import os

# label_folder = '/content/drive/MyDrive/ims'

# for filename in os.listdir(label_folder):
#     if filename.endswith('.txt'):
#         file_path = os.path.join(label_folder, filename)
#         with open(file_path, 'r') as f:
#             lines = f.readlines()
#         new_lines = []
#         found_folding = False
#         for line in lines:
#             parts = line.strip().split()
#             if 'folding' in parts:
#                 found_folding = True
#                 parts = [p for p in parts if p != 'folding']
#             new_lines.append(' '.join(parts))
#         if found_folding:
#             with open(file_path, 'w') as f:
#                 f.write('\n'.join(new_lines))
#             print(f"Updated: {filename}")

In [16]:
import os
import shutil
import random
from pathlib import Path

# Set your  directories
images_dir = Path('/root/.cache/kagglehub/datasets/zhangyunsheng/defects-class-and-location/versions/1/images/images')
labels_dir = Path('/content/drive/MyDrive/ims')

# Set output directories
output_dir = Path('datanew1')
train_dir = output_dir / 'train'
val_dir = output_dir / 'val'

# Split ratio
split_ratio = 0.6

# Gather label files

label_files = sorted([f for f in labels_dir.iterdir() if f.is_file()])

# Shuffle and split
random.shuffle(label_files)
split_idx = int(len(label_files) * split_ratio)
train_labels = label_files[:split_idx]
val_labels = label_files[split_idx:]

def copy_data(label_list, split_folder):
    (split_folder / 'labels').mkdir(parents=True, exist_ok=True)
    (split_folder / 'images').mkdir(parents=True, exist_ok=True)
    for label_path in label_list:
        shutil.copy(label_path, split_folder / 'labels' / label_path.name)
        base_name = label_path.stem
        found = False
        for class_folder in images_dir.iterdir():
            if class_folder.is_dir():
                image_path = class_folder / f"{base_name}.jpg"
                if image_path.exists():
                    shutil.copy(image_path, split_folder / 'images' / image_path.name)
                    found = True
                    break
        if not found:
            print(f"Image for label {label_path.name} not found.")

copy_data(train_labels, train_dir)
copy_data(val_labels, val_dir)


In [17]:
def create_data_yaml(train_dir, val_dir, class_map, save_path='data.yaml'):
    """Creates a data.yaml file for YOLO training."""
    data = {
        'train': train_dir,
        'val': val_dir,
        'nc': len(class_map),
        'names': list(class_map.keys())
    }
    with open(save_path, 'w') as f:
        yaml.dump(data, f)
    print(f"data.yaml created at {save_path}")



create_data_yaml(
    train_dir='/content/datanew1/train',
    val_dir='/content/datanew1/val',
    class_map=class_map,
    save_path='datanew1.yaml'
)

data.yaml created at datanew1.yaml


In [18]:
!pip install ultralytics==8.3.155
!pip install ultralytics-thop==2.0.14


In [ ]:
import torch

from ultralytics import YOLO


class YOLOv11Trainer:

    def __init__(self, model_path, data_yaml, device):

        self.device = 'cpu'# 'cuda:0'#device if torch.cuda.is_available() else "cpu"

        self.model = YOLO(model_path)

        self.data_yaml = data_yaml

    def train(self, epochs=1, imgsz=480):

        train_results = self.model.train(

            data=self.data_yaml,

            epochs=epochs,

            imgsz=imgsz,

            device=self.device

        )

        return self.model



trainer = YOLOv11Trainer(model_path="yolo11n.pt", data_yaml=f"datanew1.yaml", device='cpu')

model = trainer.train(epochs=10, imgsz=[640,312])



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 61.5MB/s]


New https://pypi.org/project/ultralytics/8.3.156 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.155 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datanew1.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=[640, 312], int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer

100%|██████████| 755k/755k [00:00<00:00, 14.0MB/s]

Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytic

...

val: Scanning /content/datanew1/val/labels... 912 images, 0 backgrounds, 0 corrupt: 100%|██████████| 912/912 [00:02<00:00, 305.12it/s]

val: New cache created: /content/datanew1/val/labels.cache


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G      2.674      7.251      2.147         14        640:  30%|███       | 26/86 [06:06<14:05, 14.10s/it]


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict('Tuned_YOLO11n.pth'))

In [19]:
import os
import shutil

# Define source directories
train_dir = '/content/datanew1/train/images'
val_dir = '/content/datanew1/val/images'

# Define destination directory
all_images_dir = 'datasetimages'

# Create destination directory if it doesn't exist
os.makedirs(all_images_dir, exist_ok=True)

# Supported image extensions
image_extensions = {'.png', '.jpg', '.jpeg', '.bmp', '.gif'}

# Function to copy images from source to destination with renaming duplicates
def copy_images(src_dir, dest_dir):
    for root, _, files in os.walk(src_dir):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            if ext in image_extensions:
                src_path = os.path.join(root, file)
                dest_path = os.path.join(dest_dir, file)
                # Rename if file exists
                count = 1
                while os.path.exists(dest_path):
                    name, ext = os.path.splitext(file)
                    new_name = f"{name}_{count}{ext}"
                    dest_path = os.path.join(dest_dir, new_name)
                    count += 1
                shutil.copy2(src_path, dest_path)

# Copy images from train and val folders
copy_images(train_dir, all_images_dir)
copy_images(val_dir, all_images_dir)

print('All images have been copied to the folder:', all_images_dir)

All images have been copied to the folder: datasetimages


Intergration with Fiftyone


In [22]:
!pip install fiftyone

  Using cached fiftyone-1.5.2-py3-none-any.whl.metadata (23 kB)
  Using cached argcomplete-3.6.2-py3-none-any.whl.metadata (16 kB)
  Using cached async_lru-2.0.5-py3-none-any.whl.metadata (4.5 kB)
  Using cached boto3-1.38.38-py3-none-any.whl.metadata (6.6 kB)
  Using cached dacite-1.7.0-py3-none-any.whl.metadata (14 kB)
  Using cached Deprecated-1.2.18-py2.py3-none-any.whl.metadata (5.7 kB)
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached hypercorn-0.17.3-py3-none-any.whl.metadata (5.4 kB)
  Using cached kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl.metadata (15 kB)
  Using cached mongoengine-0.29.1-py3-none-any.whl.metadata (6.7 kB)
  Using cached motor-3.6.1-py3-none-any.whl.metadata (21 kB)
  Using cached pprintpp-0.4.0-py2.py3-none-any.whl.metadata (7.9 kB)
  Using cached pymongo-4.9.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (22 kB)
  Using cached retrying-1.3.4-py3-none-any.whl.metadata (6.9 kB)
  Using cached rtree-1.4.0-

In [9]:
train: "/content/datanew1/train/images"
val: "/content/datanew1/val/images"

nc: 10
names: [ 'crease', 'crescent_gap', 'inclusion',
    'oil_spot', 'punching_hole', 'rolled_pit',
    'silk_spot', 'waist_folding', 'water_spot',
    'welding_line']

In [20]:
def create_data_yaml(train_dir, val_dir, class_map, save_path='data.yaml'):
    """Creates a data.yaml file for YOLO training."""
    data = {
        'train': train_dir,
        'val': val_dir,
        'nc': len(class_map),
        'names': list(class_map.keys())
    }
    with open(save_path, 'w') as f:
        yaml.dump(data, f)
    print(f"data.yaml created at {save_path}")



create_data_yaml(
    train_dir='/content/datanew1/train/images',
    val_dir='/content/datanew1/val/images',
    class_map=class_map,
    save_path='dataset.yaml'
)

data.yaml created at dataset.yaml


In [23]:
import fiftyone as fo

name = "sample_dataset1" #sample name for the dataset
dataset_dir = "/content/" #location where the dataset.yaml file is, name has to be dataset.yaml

# The splits to load #this should match the spilts defined in the dataset.yaml
splits = ["train", "val"]

# Load the dataset, using tags to mark the samples in each split
dataset = fo.Dataset(name)
for split in splits:
    dataset.add_dir(
        dataset_dir=dataset_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        tags=split,
)

 100% |███████████████| 1368/1368 [5.6s elapsed, 0s remaining, 427.4 samples/s]       


INFO:eta.core.utils: 100% |███████████████| 1368/1368 [5.6s elapsed, 0s remaining, 427.4 samples/s]       


 100% |█████████████████| 912/912 [2.0s elapsed, 0s remaining, 447.8 samples/s]         


INFO:eta.core.utils: 100% |█████████████████| 912/912 [2.0s elapsed, 0s remaining, 447.8 samples/s]         


In [1]:
from ultralytics import YOLO


model = YOLO("/content/drive/MyDrive/best.pt")

results = model.predict("/content/datasetimages", save_txt=True, save_conf=True)

ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
import os
import fiftyone.utils.yolo as fouy
import eta.core.utils as etau
labels_path = "/content/Fiftyone-yolov5-tutorial/predictions" # this could have both text files and labels

labels_dict = {
    os.path.splitext(os.path.basename(p))[0] + ".jpg": p  # assumes your images are jpg
    for p in etau.list_files(labels_path, abs_paths=True, recursive=True)
    if p.endswith(".txt")
}

fouy.add_yolo_labels(
    sample_collection=dataset,
    label_field="predictions",
    labels_path=labels_dict,
)